In [2]:
import open3d as o3d
import numpy as np
import math
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def write_pcd(file, out_file='pcd.txt', height=0):
    """ write one measurement per angle to out file (in euclidean format) """
    angles = []
    distances = []
    seen = set()

    with open(file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            angle, dist, quality = map(lambda x : float(x.split(':')[1]), line.split())
            angle = math.floor(angle)
            if angle not in seen:
                angles.append(math.radians(angle))
                distances.append(dist)
                seen.add(angle)

    angles = np.array(angles)
    distances = np.array(distances)
    
    points_x = np.cos(angles) * distances
    points_y = np.sin(angles) * distances

    # write in x y z format for open3d
    with open(out_file, 'w') as f:
        for x, y in zip(points_x, points_y):
            f.write(f'{x} {y} {height}\n')



#### Convert files to required xyz format

In [4]:
BASE_PATH_RAW_POINTS = "../data/out"
BASE_PATH_POINTS = "../data/out_uni"

#### Convert to required format

In [5]:
write_pcd(f'{BASE_PATH_RAW_POINTS}/1.txt', out_file=f'{BASE_PATH_POINTS}/1.txt')
write_pcd(f'{BASE_PATH_RAW_POINTS}/2.txt', out_file=f'{BASE_PATH_POINTS}/2.txt')
write_pcd(f'{BASE_PATH_RAW_POINTS}/3.txt', out_file=f'{BASE_PATH_POINTS}/3.txt')
write_pcd(f'{BASE_PATH_RAW_POINTS}/4.txt', out_file=f'{BASE_PATH_POINTS}/4.txt')


#### Load point clouds

In [6]:
pcd_1 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/1.txt", format='xyz')
pcd_2 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/2.txt", format='xyz')
pcd_3 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/3.txt", format='xyz')
pcd_4 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/4.txt", format='xyz')

#### Point to Point ICP

In [7]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

In [8]:
pcd_1_copy = copy.deepcopy(pcd_1)
pcd_2_copy = copy.deepcopy(pcd_2)
pcd_3_copy = copy.deepcopy(pcd_3)
pcd_4_copy = copy.deepcopy(pcd_4)


In [9]:
# this is our base scan, we are matching all other scans to this
target = pcd_1_copy
# other point clouds
sources = [pcd_2_copy, pcd_3_copy, pcd_4_copy]

# initial transformation guess
trans_init = np.eye(4,4)

for source in sources:

    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, 
        target, 
        200, 
        trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000)
    )

    print(reg_p2p)
    print("Transformation is:")
    print(reg_p2p.transformation)

    source.transform(reg_p2p.transformation)


RegistrationResult with fitness=9.972222e-01, inlier_rmse=1.462995e+01, and correspondence_set size of 359
Access transformation to get result.
Transformation is:
[[ 9.99348867e-01  3.60810385e-02  0.00000000e+00  3.88657061e+01]
 [-3.60810385e-02  9.99348867e-01  0.00000000e+00  1.61643649e+01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.493350e+01, and correspondence_set size of 360
Access transformation to get result.
Transformation is:
[[  0.97493107   0.2225071    0.          13.56550903]
 [ -0.2225071    0.97493107   0.         -61.92267342]
 [  0.           0.           1.           0.        ]
 [  0.           0.           0.           1.        ]]
RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.657564e+01, and correspondence_set size of 360
Access transformation to get result.
Transformation is:
[[  0.98285917  -0.18

In [10]:
def visualise(pcd_1, pcd_2, pcd_3, pcd_4, pcd_1_copy, pcd_2_copy, pcd_3_copy, pcd_4_copy):
    # Original point cloud
    pcd_1.translate((0,0,600))
    pcd_2.translate((0,0,1000))
    pcd_3.translate((0,0,1400))
    pcd_4.translate((0,0,1800))

    pcd_1.paint_uniform_color([0.8, 0, 0])
    pcd_2.paint_uniform_color([0, 0.8, 0])
    pcd_3.paint_uniform_color([0, 0, 0.8])
    pcd_4.paint_uniform_color([1, 0.706, 0])


    # matched point cloud
    pcd_1_copy.translate((6000,0,600))
    pcd_2_copy.translate((6000,0,1000))
    pcd_3_copy.translate((6000,0,1400))
    pcd_4_copy.translate((6000,0,1800))

    pcd_1_copy.paint_uniform_color([0.8, 0, 0])
    pcd_2_copy.paint_uniform_color([0, 0.8, 0])
    pcd_3_copy.paint_uniform_color([0, 0, 0.8])
    pcd_4_copy.paint_uniform_color([1, 0.706, 0])

    o3d.visualization.draw_geometries(
        [
            pcd_1_copy, pcd_2_copy, pcd_3_copy, pcd_4_copy,
            pcd_1, pcd_2, pcd_3, pcd_4
        ]
    )



In [11]:
visualise(pcd_1, pcd_2, pcd_3, pcd_4, pcd_1_copy, pcd_2_copy, pcd_3_copy, pcd_4_copy)

### Coherent point drift

In [28]:
pcd_1 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/1.txt", format='xyz')
pcd_2 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/2.txt", format='xyz')
pcd_3 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/3.txt", format='xyz')
pcd_4 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/4.txt", format='xyz')

In [33]:
import copy
from probreg import cpd

pcd_1_copy = copy.deepcopy(pcd_1)
pcd_2_copy = copy.deepcopy(pcd_2)
pcd_3_copy = copy.deepcopy(pcd_3)
pcd_4_copy = copy.deepcopy(pcd_4)

# this is our base scan, we are matching all other scans to this
target = pcd_1_copy
# other point clouds
sources = [pcd_2_copy, pcd_3_copy, pcd_4_copy]

for source in sources:

# compute cpd registration
    tf_param, _, _ = cpd.registration_cpd(source, target)
    source.points = tf_param.transform(source.points)

    # draw result
    source.paint_uniform_color([1, 0, 0])
    target.paint_uniform_color([0, 1, 0])
    o3d.visualization.draw_geometries([source, target])

In [ ]:
visualise(pcd_1, pcd_2, pcd_3, pcd_4, pcd_1_copy, pcd_2_copy, pcd_3_copy, pcd_4_copy)